## Launching a run

In [ ]:
from hpo_client import HyperparameterTuner
from utils import load_config

config = load_config()

### (Optional) Iterate on the objective

The following piece of code is an example `objective` function. 
You can use the [`%%writefile` magic](https://ipython.readthedocs.io/en/stable/interactive/magics.html#cellmagic-writefile) to write the file.

In [ ]:
%%writefile objective_fn_dev.py

from sklearn.datasets import load_iris
from sklearn.tree import ExtraTreeClassifier
from sklearn.model_selection import cross_val_score
import numpy as np

def objective(trial):
    data = load_iris()
    X, y = data['data'], data['target']
    max_depth = trial.suggest_int('max_depth', 4, 8)
    criterion = trial.suggest_categorical(
        'criterion',
        ["gini", "entropy"]
    )
    model = ExtraTreeClassifier(max_depth=max_depth,
                                criterion=criterion)
    return np.mean(cross_val_score(model, X, y, cv=5))

### Run the job

Now using, the `HyperparameterTuner` class defined in `hpo_client.py`, we can pass this function we are iterating on. 
The purpose of this setup is to make it easier to iterate on the contents of the objective in notebook, save it, and run the flow with minimal moving parts.

In [ ]:
tuner = HyperparameterTuner(
    # objective_function_file=config['objective_function_file'],
    objective_function_file='objective_fn_dev.py',
    override_study_name=config.get('study_name', None),
    optuna_app_name=config['optuna_app_name']
)

run_id = tuner.run_blocking(
    override_compute_pool=config['compute_pool'],
    n_trials=config['n_trials'],
    trials_per_task=config['trials_per_task']
)

## Analysis

After flows complete, the results of the hyperparameter tuning process will be captured in the `results` artifact. 
If you want to call `utils.load_study` it requires running in a Metaflow task or running the notebook/script from an Outerbounds workstation.

In [ ]:
from metaflow import Run
from utils import extract_flow_name

flow_name = extract_flow_name('flow.py', sanitize=False)
run = Run(f"{flow_name}/{run_id}")
study_df = run.data.results

In [ ]:
study_df

## Cleanup

Either remove the dev file after iteration, or replace the previous objective function definition with this one. 
How this interacts with branching in GitHub is a matter of preference.

In [ ]:
# Option A
# ! rm ./objective_fn_dev.py

# Option B
# ! cp ./objective_fn_dev.py ./objective_fn.py